In [245]:
from PIL import Image
from IPython.display import display
import random
import json
from functools import reduce
from math import log

In [246]:
# READ IN TRAITS DATA FROM JSON AND CREATE CONSTANTS

# trait data consists of two arrays : [trait_names, trait_rarity_percentage]
TRAIT_DATA = None 
with open("traits.json", "r") as rf:
    TRAIT_DATA = json.load(rf)

TRAIT_TYPES = ["Background", "Class", "Body", "Head", "Eyes", "Mouth", "Back"]
COMMON_TRAIT_TYPES = TRAIT_TYPES[2:]

In [247]:
# normalize rarities to be % out of 100
def normalize(normalized, trait_type):
    if(trait_type not in COMMON_TRAIT_TYPES):
        rarities = TRAIT_DATA[trait_type][1]
        total = reduce(lambda tot, x: tot + x, rarities)
        normalized[trait_type] = list(map(lambda x: round(x * 100 / total, 3), rarities))
    else:
        normalized[trait_type] = {}
        for class_name in ["Dragon", "Golem", "Humanoid", "Phantom"]:
            rarities = TRAIT_DATA[trait_type][class_name][1]
            total = reduce(lambda tot, x: tot + x, rarities)
            normalized[trait_type][class_name] = list(map(lambda x: round(x * 100 / total, 3), rarities))
    return normalized

normalized_rarities = reduce(normalize, TRAIT_TYPES, {})

for type in TRAIT_TYPES:
    if(type not in COMMON_TRAIT_TYPES):
        TRAIT_DATA[type][1] = normalized_rarities[type]
    else:
        for class_name in ["Dragon", "Golem", "Humanoid", "Phantom"]:
            TRAIT_DATA[type][class_name][1] = normalized_rarities[type][class_name]
    
with open('traits.json', 'w') as f:
    json.dump(TRAIT_DATA, f, indent=2)

In [248]:
# utilities
def isHumanoid(class_name): return class_name not in ["Dragon", "Golem"]
def isPhantom(class_name): return class_name == "Phantom"
def get_resolved_class(class_name): return "Humanoid" if isHumanoid(class_name) else class_name

<center><h1>Asset Path Resolver</h1></center>

In [249]:
# static paths
ASSET_PATH = "./assets/images/"
# use this as a default when none trait
NONE_PATH = "./assets/images/none.png"
# background uses a centralized folder
def get_bg_path(bg_name): return f"{ASSET_PATH}/background/{bg_name.lower()}.png"
# classname only matters for things that are humanoid
def get_class_path(class_name): return f"{ASSET_PATH}/humanoid/class/{class_name.lower()}.png" if isHumanoid(class_name) else None;
# general utility for getting asset image path
def get_asset_path(traits, trait_type):
    # return none default if trait is "none"
    if traits[trait_type] == "None": return NONE_PATH
    # format trait name to match image file names 
    formatted_trait = traits[trait_type].replace(" ", "_").lower()
    # the class path needs to be resolved
    resolved_class_path = "humanoid" if isHumanoid(traits['Class']) else traits['Class'].lower()
    return f"{ASSET_PATH}/{resolved_class_path}/{trait_type.lower()}/{formatted_trait}.png"

<center><h1>Generating File Name</h1></center>

In [250]:
def generate_file_name(traits):
    # for each trait get its index in the json object and pad with one '0'
    background_idx = TRAIT_DATA["Background"][0].index(traits["Background"])
    class_idx = TRAIT_DATA["Class"][0].index(traits["Class"])
    resolved_class_name = get_resolved_class(traits["Class"]) if not isPhantom(traits["Class"]) else "Phantom"
    common_trait_idxs = map(lambda trait: TRAIT_DATA[trait][resolved_class_name][0].index(traits[trait]), COMMON_TRAIT_TYPES)
    path_name = f"{traits['Power']}{background_idx:02d}{class_idx:02d}"
    for idx in common_trait_idxs:
        path_name = path_name + f"{idx:02d}"
    return path_name

<center><h1>Trait Resolver</h1></center>

In [251]:

def get_random(data): return random.choices(data[0],data[1])[0]
def get_random_trait(type, class_name = None): 
    # get random background or class
    if(class_name == None): return get_random(TRAIT_DATA[type])
    # get resolved class name to find trait in trait data
    resolved_class = get_resolved_class(class_name)
    # get random trait
    trait = get_random(TRAIT_DATA[type][resolved_class])
    return trait

<center><h1>Power Level</h1></center>

In [773]:
# calculate power contribution for a single trait
def calc_power(traits, type, weight):
    resolved_class = get_resolved_class(traits["Class"]) if not isPhantom(traits["Class"]) else "Phantom"
    trait_name = traits[type]
    # retrieve the rarity for the given trait
    # need to find the index of the trait name 
    trait_idx = TRAIT_DATA[type][resolved_class][0].index(trait_name) if type in COMMON_TRAIT_TYPES else TRAIT_DATA[type][0].index(trait_name)
    # use idx to index the rarities array
    trait_rarity =  TRAIT_DATA[type][resolved_class][1][trait_idx] if type in COMMON_TRAIT_TYPES else TRAIT_DATA[type][1][trait_idx]
    # dampening factor (use log to make sure numbers remain somewhat closer together at scale)
    # add 1 in order to ensure dampen is never 0
    dampen =  (1 + abs(log(trait_rarity))) / 2
    # dampen the weight that was given for this trait, 
    # the dampening factor will be smaller if the rarity is "high" AKA low percentage => higher power
    return weight / dampen

In [774]:
def get_power(traits):
    resolved_class = get_resolved_class(traits["Class"]);
    is_humanoid = isHumanoid(traits["Class"])
    is_golem = resolved_class == "Golem"
    
    # Create weights for each trait that are multipliers to its power
    # each trait has separate scaling factors based on the class to balance power levels
    TRAIT_WEIGHTS = [
        500,                                                    # background
        3000,                                                   # class
        1800 * (1 if is_humanoid else 2),                       # body
        1200 * (1 if is_humanoid else 2 if is_golem else 1.5),  # head
        800 * (1 if is_humanoid else 1.6),                      # eyes
        650,                                                    # mouth
        200 * (1 if is_humanoid else 2 if is_golem else 1.6)    # back
    ]
    
    # calculate the power contribution for each trait
    trait_powers = [calc_power(traits, type, weight) for type, weight in zip(TRAIT_TYPES, TRAIT_WEIGHTS)]
    
    # sum powers and round
    power = round(reduce(lambda curr, tot: curr + tot, trait_powers))
    
    return power
    

<center><h1>Generate Trait Metadata</h1></center>

In [852]:
def generate_metadata(n = 200):
    metadata = []
    # used for checking if metadata already exists easily
    filenames = []
    i = 0
    while i < n:
        # generate background and class first since they are exceptions
        new_background = get_random_trait("Background")
        new_class = get_random_trait("Class")
        # generate rest of traits
        potr_metadata = {
            "Background": new_background,
            "Class": new_class,
            "Body": get_random_trait("Body", new_class),
            "Head": get_random_trait("Head",new_class),
            "Eyes": get_random_trait("Eyes",new_class),
            "Mouth": get_random_trait("Mouth",new_class),
            "Back":  get_random_trait("Back",new_class)
        }
        
        # retrieve the power for these traits
        power = get_power(potr_metadata)
        # add power to traits and add it to list of metadata
        potr_metadata["Power"] = power
        
        # loop again if these traits exist
        if(generate_file_name(potr_metadata) in filenames):
            continue;
        else:
            i += 1
            filenames.append(generate_file_name(potr_metadata))
            metadata.append(potr_metadata)
    
    # return metadata once all of the requested objects have been made
    return metadata

<center><h1>------------- GENERATE POTRS HERE -------------</h1></center>

In [853]:
# n = # of nfts to makew
NUM_POTRS = 6000
potr_traits = generate_metadata(n = NUM_POTRS)

Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Match found
Matc

In [854]:
def get_trait_count(count, curr_traits): 
    for trait_type in TRAIT_TYPES:
        trait_name = curr_traits[trait_type]
        
        # init count for trait_type
        if(trait_type not in count): count[trait_type] = {}
        
        # init count for trait
        if(trait_name not in count[trait_type]):
            count[trait_type][trait_name] = 1 
            continue;

        # increment
        count[trait_type][trait_name] += 1
    return count

def get_power_count(count, curr_traits): 
    # get power
    power = curr_traits["Power"]
    
    # init count for power
    if(power not in count): count[power] = 0

    # increment
    count[power]+= 1
    
    return count
        
def get_trait_stats(stats, trait_type):
    # iterate over trait names in category
    for trait_name, count in stats[trait_type].items():
        # calculate the rarity fraction of trait
        true_rarity = round(count * 100 / NUM_POTRS, 3)
        # add true rarity to stats
        stats[trait_type][trait_name] = [count, true_rarity]
    return stats

In [866]:
trait_counts = reduce(get_trait_count, potr_traits, {})
trait_stats = reduce(get_trait_stats, TRAIT_TYPES, trait_counts)

# sort by descending power if needed
potr_traits.sort(reverse=True, key=(lambda traits: traits["Power"]))

power_counts = reduce(get_power_count, potr_traits, {})
print(reduce(lambda max, power: [power, power_counts[power]] if power_counts[power] > max[1] else max, power_counts, [0,0]))
print(reduce(lambda max, power: power if power > max else max, power_counts))


# write traits to json if needed
with open('metadata.json', 'w') as f:
    json.dump(potr_traits, f, indent=2)
    
with open('stats.json', 'w') as f:
    json.dump(trait_stats, f, indent=2)

[4674, 12]
10138


In [865]:
filenames = [generate_file_name(traits) for traits in potr_traits]
print(len(potr_traits))
print(len(filenames))
print(len(set(filenames)))

6000
6000
6000


<center><h1>Generate Images</h1></center>

In [860]:
def generate_layers(traits):
    layers = {
        "Background": Image.open(get_bg_path(traits["Background"])).convert('RGBA'),
        "Body": Image.open(get_asset_path(traits, "Body")).convert('RGBA'),
        "Head": Image.open(get_asset_path(traits, "Head")).convert('RGBA'),
        "Eyes": Image.open(get_asset_path(traits, "Eyes")).convert('RGBA'),
        "Mouth": Image.open(get_asset_path(traits, "Mouth")).convert('RGBA'),
        "Back": Image.open(get_asset_path(traits, "Back")).convert('RGBA'),
    }
    return layers
def generate_dragon_layers(traits):
    back_trait = traits["Body"] if traits["Back"] == "Dragon Wings" else traits["Back"]
    traits["Back"] = back_trait
    layers = generate_layers(traits)   
    layers["Class"] = Image.open(NONE_PATH).convert('RGBA')
    if(traits["Back"] not in TRAIT_DATA["Back"]["Dragon"][0]): traits["Back"] = "Dragon Wings"
    return layers
def generate_golem_layers(traits):
    layers = generate_layers(traits)   
    layers["Class"] = Image.open(NONE_PATH).convert('RGBA')
    return layers
def generate_humanoid_layers(traits):
    layers = generate_layers(traits)
    layers["Class"] = Image.open(get_class_path(traits["Class"])).convert('RGBA')
    return layers

In [861]:
#  this takes the layers and creates a 
def create_image_composite(traits, layers):
    trait_types = TRAIT_TYPES.copy()
    
    # if not humanoid remove class so we dont add an unnecessary layer for class trait
    if not isHumanoid(traits["Class"]): trait_types.remove("Class")
    
    # create first layer
    potr = Image.alpha_composite(layers["Background"], layers["Back"]);
    
    # remove used layers
    trait_types.remove("Background")
    trait_types.remove("Back")
    
    # add rest of layers
    for type in trait_types:
        potr = Image.alpha_composite(potr, layers[type])
        
    # Convert to RGB
    potr = potr.convert('RGB')
    return potr
            

<center><h1>------------- GENERATE IMAGES HERE -------------</h1></center>

In [867]:
for traits in potr_traits:
    class_name = traits["Class"]
    
    layers = None;
    if(class_name == "Dragon"):
        layers = generate_dragon_layers(traits)
    elif(class_name == "Golem"):
        layers = generate_golem_layers(traits)
    else:
        layers = generate_humanoid_layers(traits)

    # create composite images
    potr = create_image_composite(traits, layers)

    # generate file name based on traits
    potr_name = generate_file_name(traits)
    
    # save the new image file
    potr.save(f"./nfts/{potr_name}.png")